**April 17, 2014**
 
**@author B.F. Habenicht (bfh)**


Script to extract MO coefficients, dipole matrix, and CI coefficients from Gaussian simulation and use these to calculate the transition dipole moments between all pairs of states. these will later be used by the python script TD_CIS.py to do excited state electron dynamics in an electric field. we first find the data in the gaussian log file. we then rotate the dipole matrix into the MO basis (from AO basis) using the MO coefficient matrix. once we have this, we then can multiply the appropriate CI coefficients together and multiply those terms times the dipole matrix in the MO basis. These terms are collected and summed to calculate the transition dipole moments between states.

**Important notes**

- For a CASSCF calculation with only two electrons in the active space, the subsequently determined configurations are not spin-adapted. 

- Having `CAS(SaveGEDensities)` in the route section means that the configurations determined by `Gaussian`'s MC-SCF subroutine are not spin-adapted any more even if the number of electrons is more than two and the reference is restricted.

\- KR, 05/17/2020

In [1]:
import sys
import numpy as np
import os
import shutil
import re
import time
sys.path.append('/mnt/c/Users/karna/Github/kranka_ucm/scripts/')
sys.path.append('/mnt/c/Users/kranka/Github/kranka_ucm/scripts/')
sys.path.append('/mnt/d/Github/kranka_ucm/scripts/')
from gauss_hf import *

**Defining conversion factors and parameters**

In [2]:
# dipole moment conversion factor: 1 debye = 0.393430307 a.u.
DtoAU = 0.39343
AUtoD = 1./DtoAU

# energy conversion factor: 1 a.u. = 27.211396 eV
AUtoEV = 27.211396
EVtoAU = 1./AUtoEV

# define square root of 2 - bfh
rt2 = np.sqrt(2)

In [3]:
!rm dens*.npz

In [4]:
#
# Route section for CASSCF calculation using "GDV i14+":
#   #P CASSCF(4,6,fulldiag,NRoot=225,SaveGEDensities)/sto-3g
#      scf(tight,maxcyc=150) nosymm iop(6/8=1,3/33=3,3/36=1,4/33=3,5/33=3,
#      6/33=3,9/33=3,5/72=-3)
#

# Logfile = './casscf46_s225_lih_sto-3g.log'
#Logfile = './logfiles/casscf24_s15_heh+_6-31g.log'
#Logfile = './logfiles/casscf22_s2_heh+_sto-3g.log'
#Logfile = './logfiles/casscf22_s2_h2_sto-3g.log'
Logfile = './logfiles/casscf24_s15_h2_6-31g.log'

Log = Logfile.split('/')[-1].split('.')[0]
terms = Logfile.split('/')[-1].split('_')

ci_str = list(terms[0])
# print(ci_str)
if ci_str[1] == 'a':
    ci = 'casscf'
else:
    print('not the right .LOG file for this code.')
    quit()

#specify CAS here
cas = ''.join(ci_str[6:])
sN = terms[-3]
basis = terms[-1].split('.')[0]
molecule = terms[-2]
ext = '.'+terms[-1].split('.')[1]

In [5]:
debug = 1
if debug == 1:
    print('file input:', Logfile)
    print('detecting whether file exists...')
    if os.path.isfile(Logfile) == True:
        print('file exists.')
    elif os.path.isfile(Logfile) == False:
        print('error: file does not exist.')
debug = 0

file input: ./logfiles/casscf24_s15_h2_6-31g.log
detecting whether file exists...
file exists.


In [6]:
# read the .LOG file after copying it to 'logfile.tmp'
outp = 'logfile.tmp'
shutil.copy(Logfile, outp)

datafile = log_data(outp)
log_lines = datafile.loglines


Reading data from logfile.tmp...
	Gaussian .LOG data read.



**Defining/reading parameters**

In [7]:
# read parameters from the Gaussian .LOG file of CASSCF calculation
#  (currently works for restricted reference, minimum spin configuration only)
#  - KR, 05/17/2020
for (n,line) in enumerate(log_lines):
    try:
        NMOs = datafile.nao        # total number of basis fns/MOs with double occupancy
        NELECT_A = datafile.n_a    # number of alpha electrons in the system
        NELECT_B = datafile.n_b    # number of beta electrons in the system
        NOCC = NELECT_A            # total number of occupied MOs in the reference
        if ('NAtoms' in line):
            elements = log_lines[n].split()
            NAtoms = int(float(elements[1]))      # total numer of atoms in the system
        if ('NO OF BASIS FUNCTIONS' in line):
            elements = log_lines[n].split()
            NStates = int(float(elements[5]))     # number of CI states/CSFs
            print('for "CASSCF(NRoot=N)" option, please make sure N is the same as the \n total no. of configurations, NStates\n')
        if ('NO. OF ORBITALS' in line):
            elements = log_lines[n].split('=')
            NCAS = int(float(elements[1]))        # number of doubly occupied MOs in the active space
        if ('NO. OF ELECTRONS' in line):
            elements = log_lines[n].split('=')
            NELECT = int(float(elements[1]))      # total number of electrons in the active space
            if ((NELECT % 2) == 1):
                NELECT_CAS_A = int((NELECT-1)/2)
                NELECT_CAS_A += 1                 # number of alpha electrons in the active space
                NELECT_CAS_B = NELECT - NELECT_CAS_A
            else:
                NELECT_CAS_A = int(NELECT/2)
                NELECT_CAS_B = NELECT_CAS_A
            NFRZ = NELECT_A - NELECT_CAS_A        # number of doubly occupied MOs outside of the active space
    except (ValueError, IndexError, TypeError, NameError):
        print('Error encountered while reading parameters.')
        break

for "CASSCF(NRoot=N)" option, please make sure N is the same as the 
 total no. of configurations, NStates



In [8]:
debug = 2
if debug == 1:
    print('CAS({},{}) calculation...'.format(NELECT, NCAS))
    print('total no. of MOs with double occupancy:\t\t {}'.format(NMOs))
    print('no. of occupied MOs with frozen configuration:\t {}'.format(NFRZ))
    print('total no. of occupied MOs:\t\t\t {}'.format(NOCC))
    print('total no. of atoms in the system:\t\t {}'.format(NAtoms))
    print('total no. of determinants in the system:\t {}'.format(NStates))
elif debug == 2:
    print('NMOs \t= {}'.format(NMOs))
    print('NCAS \t= {}'.format(NCAS))
    print('NFRZ \t= {}'.format(NFRZ))
    print('NOCC \t= {}'.format(NOCC))
    print('NELECT \t= {}'.format(NELECT))
    print('NAtoms \t= {}'.format(NAtoms))
    print('NStates = {}'.format(NStates))
debug = 0

NMOs 	= 4
NCAS 	= 4
NFRZ 	= 0
NOCC 	= 1
NELECT 	= 2
NAtoms 	= 2
NStates = 16


**Reading nuclear dipole moments**

In [9]:
# reading nuclear dipole moments from .LOG file
for (n,line) in enumerate(log_lines):
    try:
        if ('Nuclear    moments (au)' in line):
            elements = log_lines[n+1].split()
            nuc_dipx = float(elements[1])      # nuclear dipole moment along x-axis of the system in the input
            nuc_dipy = float(elements[2])      # nuclear dipole moment along y-axis of the system in the input
            nuc_dipz = float(elements[3])      # nuclear dipole moment along z-axis of the system in the input
            nuc_dipx *= AUtoD
            nuc_dipy *= AUtoD
            nuc_dipz *= AUtoD
    except (ValueError, TypeError, NameError):
        print('Error encountered. Assigning nuclear dipole moments manually...')
        if (molecule == 'lih'):
            nuc_dipx=0.0
            nuc_dipy=0.0
            nuc_dipz=2.89128097
        elif (molecule == 'sys1_h'):
            nuc_dipx=0.0
            nuc_dipy=0.0
            nuc_dipz=0.0
        break

In [10]:
debug = 1
if debug == 1:
    print('Nuclear dipole moment along-')
    print('{}-axis: {:3.2f} D ({:3.4f} a.u.)'.format('x',nuc_dipx,nuc_dipx*DtoAU))
    print('{}-axis: {:3.2f} D ({:3.4f} a.u.)'.format('y',nuc_dipy,nuc_dipy*DtoAU))
    print('{}-axis: {:3.2f} D ({:3.4f} a.u.)'.format('z',nuc_dipz,nuc_dipz*DtoAU))
debug = 0

Nuclear dipole moment along-
x-axis: 0.00 D (0.0000 a.u.)
y-axis: 0.00 D (0.0000 a.u.)
z-axis: 0.00 D (0.0000 a.u.)


**Reading dipole moment matrices represented in the AO basis**

In [11]:
dipX = datafile.get_dipole_x_AO()
dipY = datafile.get_dipole_y_AO()
dipZ = datafile.get_dipole_z_AO()
np.savez('./dipAO.npz', dipXAO_data=dipX, dipYAO_data=dipY, dipZAO_data=dipZ)

In [12]:
# debugging (following code by bfh)
debug = 0
if debug == 1:
    print('Dipole X')
    for i in range(NMOs):
        for j in range(NMOs):
            print("%d  %d  %f" % (i,j, dipX[i,j]))
    print('Dipole Y')
    for i in range(NMOs):
        for j in range(NMOs):
            print("%d  %d  %f" % (i,j, dipY[i,j]))
    print('Dipole Z')
    for i in range(NMOs):
        for j in range(NMOs):
            print("%d  %d  %f" % (i,j, dipZ[i,j]))
debug = 0

**Reading MO coefficients as column-vectors**

In [13]:
# specific to CASSCF calculation
MO = np.zeros([NMOs,NMOs], np.float64)

line_num = []
for (n, line) in enumerate(log_lines):
    try:
        if ('FINAL COEFFICIENT MATRIX' in line):
            line_num.append(n)
    except (IndexError, ValueError):
        pass

count = -1
nline = line_num[count]
loops = int(NMOs / 10) + 1
last = NMOs % 10
for i in range(NMOs):
    for k in range(loops):
        try:
            if (k == (loops - 1)):
                end = last
            else:
                end = 10
            dum1 = nline+1+i*(1+loops)+(k+1)
            elements=log_lines[dum1].split()
            for j in range(end):
                s = k*10 + j
                m = i
                MO[s,m] = float(elements[j])
        except (IndexError, ValueError):
            break

In [14]:
debug = 0
if debug == 1:
    print('MO coefficients (column-vectors)')
    print(MO)
debug = 0

In [15]:
# seems that this block of code is unnecessary - KR

# # need to truncate MOs so only those in occupied space (NOCC) have values - bfh
# 
# if (NOCC < NMOs):
#    for i in range(NOCC,NMOs):
#        for j in range(NOCC,NMOs):
#            MO[i][j] = 0.0
#            MO[j][i] = 0.0
#            dipX[i][j] = 0.0
#            dipX[j][i] = 0.0
#            dipY[i][j] = 0.0
#            dipY[j][i] = 0.0
#            dipZ[i][j] = 0.0
#            dipZ[j][i] = 0.0

**Transforming the dipole moment matrices from AO basis to MO basis**

*(Not necessary for the current method which works entirely within the AO basis)*

In [16]:
dipXMO = MO.T.dot(dipX).dot(MO)
dipYMO = MO.T.dot(dipY).dot(MO)
dipZMO = MO.T.dot(dipZ).dot(MO)

In [17]:
debug = 0
if (debug == 2):
    print('MO Dipole:')
    print('x:\n',dipXMO)
    print('y:\n',dipYMO)
    print('z:\n',dipZMO)
debug = 0

**Determining the dimensions of the matrix of CI vectors**

**NOTE**: The CASSCF vectors will be arranged in a square matrix of dimensions $(M \times M)$, where $M$=`NStates`, in the current case.

In [18]:
# specific to CASSCF calculation
M = NStates
N = M

**Reading CI coefficients and energies**

In [19]:
# specific to CASSCF calculation
CI_vect = np.zeros([M,N], np.float64)
CI_E = np.zeros([M], np.float64)

line_num = []
for (n, line) in enumerate(log_lines):
    if ('FINAL EIGENVALUES ANE EIGENVECTOR' in line):
        line_num.append(n)

print(line_num)
count = -1
nline = line_num[count]
#print(nline)
loops = int(M / 5) + 1
last = M % 5
shift = 0
for i in range(M):
    try:
        for k in range(loops):
            if (k == (loops - 1)):
                end = last
            else:
                end = 5
            dum1 = nline+5+i*(loops)+k+shift
            # print(log_lines[dum1-1])#),'\n',log_lines[dum1+1])
            elements=log_lines[dum1].split()
            # print(elements)
            # print(i)
            if (k == 0):
                CI_E[i] = float(elements[1])
                for j in range(end):
                    s = j + 2
                    CI_vect[i,j] = float(elements[s])
            else:
                for j in range(end):
                    s = 5*k + j
                    CI_vect[i,s] = float(elements[j])
            if (last == 0):
                if (loops < 2):
                    shift += 1
        if (last != 0):
            shift += 1
    except (IndexError):
        raise(IndexError)
        break

# fname = Log+'_CI_col-vecs.npz'
# np.savez(fname, CI_vect.T)
# x = np.load(fname, allow_pickle=True)
# print(x.files)

[1973]


In [20]:
debug = 1
if debug == 1:
    print('# CI states = ',len(CI_E))
    print('CI Eigenvalues:')
    print(CI_E[:])
elif debug == 2:
    print(CI_vect.shape)
    print('CI Eigenvalues and Eigenvectors:')
    for i in range(M):
        print(i+1, CI_E[i])
        print(CI_vect[i,:])
debug = 0

# CI states =  16
CI Eigenvalues:
[-1.1516725 -0.7569152 -0.5890775 -0.2917924 -0.1043666 -0.0406237
  0.2215958  0.2664408  0.3256891  0.6080613  0.7600205  0.8182611
  0.9544508  1.2007179  1.4676406  1.9276982]


**Reading MO configurations for each determinant in the basis**

**NOTE**: The occupation numbers (which is what comprises the "MO configurations") are calculated as a sum of weighted occupations of each orbital in the active space, the weights corresponding to the norm-squared of the CI coefficients.

In [21]:
# specific to CASSCF calculation

# initialize arrays to read and analyze MO configuration for each configuration - bfh
tmp = np.zeros((M), dtype='object')
config_int = np.zeros((M,NCAS), dtype='object')
multi = np.zeros((M), dtype='object')

for (n, line) in enumerate(log_lines):
    if ('BOTTOM WEIGHT' in line):
        for i in range(M):
            spin = 0
            spin_a = 0
            spin_b = 0
            elements = log_lines[n+i+1].split()
            tmp = list(elements[4])
            for j in range(NCAS):
                if (tmp[j] == '1'):
                    tmp[j] = '2'
                if (tmp[j] == 'a'):# or tmp[j] == 'b'):
                    tmp[j] = '1'
                    spin_a += 0.5
                if tmp[j] == 'b': 
                    tmp[j] = '1'
                    spin_b -= 0.5
                config_int[i,j] = int(tmp[j])
            spin += abs(spin_a + spin_b)
            multi[i] = int(2*spin + 1)

# print(config_int)
config = np.zeros((M,NCAS), np.float64)

for i in range(M):
    for k in range(NCAS):
        sum = 0
        for j in range(N):
            occ_coef = CI_vect[i,j]*np.conjugate(CI_vect[i,j])
            #
            # WARNING: this works for spin-unadapted configurations ONLY!!
            #
            sum += config_int[j,k] * occ_coef
        config[i,k] = sum

In [22]:
debug = 0
if (debug == 1):
    print('CAS configurations:')
    for i in range(NStates):
        print('CI state {}   '.format(i+1))
        for j in range(NCAS):
            if (j == (NCAS - 1)):
                print(config[i,j],end='\n')
            else:
                print(config[i,j],end='\t')
elif (debug == 2):
    print('CAS basis configurations and multiplicities:')
    for i in range(N):
        print('configuration {}   (multiplicity = {})   '.format(i+1,multi[i]))
        for j in range(NCAS):
            if (j == (NCAS - 1)):
                print(config_int[i,j],end='\n')
            else:
                print(config_int[i,j],end='\t')
elif (debug == 3):
    print('CAS basis configurations and multiplicities:')
    for i in range(N):
        print('configuration {}   (multiplicity = {})   '.format(i+1,multi[i]))
        for j in range(NCAS):
            if (j == (NCAS - 1)):
                print(config_int[i,j],end='\n')
            else:
                print(config_int[i,j],end='\t')
    print('CAS configurations:')
    for i in range(NStates):
        print('CI state {}   '.format(i+1))
        for j in range(NCAS):
            if (j == (NCAS - 1)):
                print(config[i,j],end='\n')
            else:
                print(config[i,j],end='\t')
debug = 0

**Calculating electric dipole moment matrix in the CI state basis**

**NOTE**: The formula used for this is: $\boldsymbol{\mu}_\text{CI}^{i} = \text{tr}(\textbf{P}_\text{CI}^\text{AO} \boldsymbol{\mu}_\text{AO}^{i})$, where, $i \in \{x, y, z\}$

In [23]:
# specific to CASSCF calculation
#
# calculating transition dipole moments b/w CASSCF ground state
#   and excited states
#
dipXCI = np.zeros([M,M], np.float64)
dipYCI = np.zeros([M,M], np.float64)
dipZCI = np.zeros([M,M], np.float64)

densCI_AO_a = np.zeros([NMOs,NMOs], np.float64)
densCI_AO_b = np.zeros([NMOs,NMOs], np.float64)

print('ground to excited state transitions...')

last = NMOs % 5
if (last == 0):
    loops = int(NMOs / 5)
else:
    loops = int(NMOs / 5) + 1
if (NCAS % 5 == 0):
    loops_cas = int(NCAS / 5)
else:
    loops_cas = int(NCAS / 5) + 1
skip_lines = loops_cas
# print(loops,loops_cas)
for i in range(loops_cas):
    skip_lines += NCAS - 5*i
# print(skip_lines)
for (n,line) in enumerate(log_lines):
    if ('MO Ground to excited state density ' in line and '(alpha):' == line.split()[-1]):
        st1 = int(float(log_lines[n-2].split()[-1])) - 1
        st2 = int(float(log_lines[n-1].split()[-1])) - 1
        elements = log_lines[n+2]
        # read the alpha and beta transition densities in AO basis
        # AOdensline_a = n+1+2+2*loops*(NMOs+1)+2+1
        # print(n+1,log_lines[n])
        AOdensline_a = n+2*(loops_cas)*(NCAS+1)+3
        # print(AOdensline_a+1,log_lines[AOdensline_a])
        for k in range(loops):
            for i in range(NMOs):
                try:
                    if (k == (loops - 1)):
                        end = last
                    else:
                        end = 5
                    dum1 = AOdensline_a+k*(1+NMOs)+i+1
                    dum2 = dum1+1+loops*(NMOs+1)
                    elements_a = log_lines[dum1].split()
                    elements_b = log_lines[dum2].split()
                    # if st2 < 4:
                    #     print(dum1+1,elements_a)
                    #     print(dum2+1,elements_b)
                    for j in range(end):
                        s = k*5 + j
                        m = i
                        densCI_AO_a[m,s] = float(elements_a[j+1])
                        densCI_AO_b[m,s] = float(elements_b[j+1])
                except (IndexError, ValueError):
                    pass
        densCI_AO = densCI_AO_a + densCI_AO_b
        if st2 < 4:
            print(st1,st2,'\n',densCI_AO,'\n')
        dipXCI[st1,st2] = -1*np.trace(np.matmul(densCI_AO,dipX))
        dipXCI[st2,st1] = dipXCI[st1,st2]
        dipYCI[st1,st2] = -1*np.trace(np.matmul(densCI_AO,dipY))
        dipYCI[st2,st1] = dipYCI[st1,st2]
        dipZCI[st1,st2] = -1*np.trace(np.matmul(densCI_AO,dipZ))
        dipZCI[st2,st1] = dipZCI[st1,st2]
        # if (st1 == 0 and st2 == 3):
        #     print(densCI_AO_a, densCI_AO_b, dipZCI[st1,st2])
        # print(st1, st2,'\n', dipZCI[st1,st2])
        dens_file = 'dens_ci_'+str(st1)+'_'+str(st2)+'.npz' 
        np.savez(dens_file, densCI_AO)

ground to excited state transitions...
0 1 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 

0 2 
 [[-0.1285912  0.239184   1.185458  -1.480612 ]
 [-0.353264   0.715444   3.2006    -4.62252  ]
 [ 0.0479658 -0.352614  -0.90846    1.054372 ]
 [-0.39199    0.338704   2.78748   -4.14538  ]] 

0 3 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 



In [24]:
#
# calculating transition dipole moments b/w CASSCF excited states
#

print('excited to excited state transitions...')

for (n,line) in enumerate(log_lines):
    if ('Alpha transition density between states' in line):
        elements = log_lines[n].split(':')[0].split()
        st1 = int(float(elements[-1])) - 1
        st2 = int(float(elements[-2])) - 1
        densCI_AO_a = np.zeros([NMOs,NMOs], np.float64)
        densCI_AO_b = np.zeros([NMOs,NMOs], np.float64)
        for k in range(loops):
            for i in range(NMOs):
                try:
                    if (k == (loops - 1)):
                        end = last
                    else:
                        end = 5
                    dum1 = n+2+k*(1+NMOs)+i
                    dum2 = dum1+1+loops*(NMOs+1)
                    elements_a = log_lines[dum1].split()
                    elements_b = log_lines[dum2].split()
                    print(st1,st2,'\n',elements_a,'\n',elements_b)
                    for j in range(end):
                        s = k*5 + j
                        m = i
                        densCI_AO_a[m,s] = float(elements_a[j+1])
                        densCI_AO_b[m,s] = float(elements_b[j+1])
                except (IndexError, ValueError):
                    break
        densCI_AO = densCI_AO_a + densCI_AO_b
        print(st1,st2,'\n',densCI_AO/2,'\n')
        dipXCI[st1,st2] = -1*np.trace(np.matmul(densCI_AO,dipX))
        dipXCI[st2,st1] = dipXCI[st1,st2]
        dipYCI[st1,st2] = -1*np.trace(np.matmul(densCI_AO,dipY))
        dipYCI[st2,st1] = dipYCI[st1,st2]
        dipZCI[st1,st2] = -1*np.trace(np.matmul(densCI_AO,dipZ))
        dipZCI[st2,st1] = dipZCI[st1,st2]
        print(st1, st2,'\n', dipZCI[st1,st2])
        dens_file = 'dens_ci_'+str(st1)+'_'+str(st2)+'.npz'
        print(st1,st2,'\n',densCI_AO_a)
        #np.savez(dens_file, densCI_AO)

excited to excited state transitions...
1 2 
 ['1', '0.119353E+00', '0.384726E+00', '-0.294914E-01', '0.255320E+00'] 
 ['1', '-0.119353E+00', '-0.384726E+00', '0.294914E-01', '-0.255320E+00']
1 2 
 ['2', '0.573791E+00', '0.165462E+01', '-0.425087E+00', '0.141188E+01'] 
 ['2', '-0.573791E+00', '-0.165462E+01', '0.425087E+00', '-0.141188E+01']
1 2 
 ['3', '-0.294914E-01', '-0.382862E+00', '-0.560317E+00', '0.297667E+00'] 
 ['3', '0.294914E-01', '0.382862E+00', '0.560317E+00', '-0.297667E+00']
1 2 
 ['4', '0.521214E+00', '0.141188E+01', '-0.666379E+00', '0.113325E+01'] 
 ['4', '-0.521214E+00', '-0.141188E+01', '0.666379E+00', '-0.113325E+01']
1 2 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 

1 2 
 -0.0
1 2 
 [[ 0.119353   0.384726  -0.0294914  0.25532  ]
 [ 0.573791   1.65462   -0.425087   1.41188  ]
 [-0.0294914 -0.382862  -0.560317   0.297667 ]
 [ 0.521214   1.41188   -0.666379   1.13325  ]]
1 3 
 ['1', '0.363342E-01', '-0.429462E-01', '-0.318321E+00', '0.296282E+00']

In [25]:
#
# calculating state dipole moments of all CASSCF states
#

print('stationary states...')

for (n,line) in enumerate(log_lines):
    if ('1st state is' in line):
        st1 = int(float(log_lines[n].split()[-1])) - 1
        st2 = int(float(log_lines[n+1].split()[-1])) - 1
        if (st1 == st2):
            print('state = {}'.format(st1))
            elements = log_lines[n+2]
            if ('MO valence' in log_lines[n+2]):
                densCI_AO_a = np.zeros([NMOs,NMOs], np.float64)
                shift = 0
                # read the alpha state densities in AO basis
                AOdensline_a = n+2*(loops_cas+1)+2*skip_lines
                # print(AOdensline_a,log_lines[AOdensline_a])
                for k in range(loops):
                    try:
                        irange = NMOs - k*5
                        for i in range(irange):
                            if (k == (loops - 1)):
                                if (i < last):
                                    end = i + 1
                                else:
                                    end = last
                            else:
                                if (i <= 4):
                                    end = i + 1
                                else:
                                    end = 5
                            dum1 = AOdensline_a+k+shift+i+2
                            elements=log_lines[dum1].split()
                            #print(elements)
                            for j in range(end):
                                s = k*5 + j
                                m = i + k*5
                                densCI_AO_a[m,s] = float(elements[j+1])
                                if (i != j):
                                    densCI_AO_a[s,m] = densCI_AO_a[m,s]
                        shift += irange
                    except (IndexError, ValueError):
                        break
                densCI_AO = densCI_AO_a
                print('denmat:\n',densCI_AO)
            dipXCI[st1,st1] = -2*np.trace(np.matmul(densCI_AO,dipX))
            dipYCI[st1,st1] = -2*np.trace(np.matmul(densCI_AO,dipY))
            dipZCI[st1,st1] = -2*np.trace(np.matmul(densCI_AO,dipZ))
            print('dipole moments of state {}: {}*x, {}*y, {}*z'.format(st1,dipXCI[st1,st1],dipYCI[st1,st1],dipZCI[st1,st1]))

        dens_file = 'dens_ci_'+str(st1)+'_'+str(st2)+'.npz'
        print(densCI_AO)
        #np.savez(dens_file, densCI_AO)

stationary states...
state = 0
denmat:
 [[0.116056  0.0914284 0.103039  0.0808488]
 [0.0914284 0.0773399 0.0808488 0.0667395]
 [0.103039  0.0808488 0.116056  0.0914284]
 [0.0808488 0.0667395 0.0914284 0.0773399]]
dipole moments of state 0: -0.0*x, -0.0*y, -2.7755575615628914e-17*z
[[0.116056  0.0914284 0.103039  0.0808488]
 [0.0914284 0.0773399 0.0808488 0.0667395]
 [0.103039  0.0808488 0.116056  0.0914284]
 [0.0808488 0.0667395 0.0914284 0.0773399]]
state = 1
denmat:
 [[ 0.117113  0.253086  0.036237 -0.170528]
 [ 0.253086  1.15203  -0.170528 -1.10668 ]
 [ 0.036237 -0.170528  0.117113  0.253086]
 [-0.170528 -1.10668   0.253086  1.15203 ]]
dipole moments of state 1: -0.0*x, -0.0*y, -2.220446049250313e-16*z
[[ 0.117113  0.253086  0.036237 -0.170528]
 [ 0.253086  1.15203  -0.170528 -1.10668 ]
 [ 0.036237 -0.170528  0.117113  0.253086]
 [-0.170528 -1.10668   0.253086  1.15203 ]]
state = 2
denmat:
 [[ 0.0967235  0.111085   0.0881385 -0.0367542]
 [ 0.111085   1.5619    -0.0367542 -1.53092  ]

In [26]:
dipZCI

array([[-2.77555756e-17, -0.00000000e+00,  1.06527736e+00,
        -0.00000000e+00, -1.57052099e-01, -5.67697630e-01,
        -0.00000000e+00,  2.64014128e+00, -0.00000000e+00,
         2.72423019e+00, -0.00000000e+00,  1.35492958e-01,
         1.75528935e-01, -0.00000000e+00,  1.49098911e-02,
        -1.98138821e-02],
       [-0.00000000e+00, -2.22044605e-16, -0.00000000e+00,
        -8.57534004e-01, -0.00000000e+00, -0.00000000e+00,
         5.50480052e-01, -0.00000000e+00,  9.87166158e-01,
        -0.00000000e+00,  1.81408416e-01, -0.00000000e+00,
        -0.00000000e+00, -1.51638935e-01, -0.00000000e+00,
        -0.00000000e+00],
       [ 1.06527736e+00, -0.00000000e+00, -4.44089210e-16,
        -0.00000000e+00, -1.08381550e-01, -2.74308707e+00,
        -0.00000000e+00, -9.71793681e-01, -0.00000000e+00,
         5.30504333e-01, -0.00000000e+00,  1.58060529e-01,
        -7.71970073e-01, -0.00000000e+00, -3.91804478e-01,
        -3.27402610e-02],
       [-0.00000000e+00, -8.57534004e

In [27]:
dipZCI.shape

(16, 16)

In [28]:
with open('casscf24_s15_h2_6-31g_CI_dimat.npz','wb') as f:
    np.save(f,dipZCI)

In [26]:
debug = 0
if debug == 1:
    print('Total number of allowed states for electric dipole transition from the ground state:\n')
    print(len(spin_allowed) - 1)
elif debug == 2:
    print('Total number of spin-allowed (singlet) states for transition from the ground state:\n')
    print(len(spin_allowed[1:]))
    print('Spin-allowed excited states:\n')
    print(spin_allowed[1:])
elif debug == 3:
    print('CI dipole moment matrices (a.u.):')
    print('along x-axis:\n', dipXCI)
    print('along y-axis:\n', dipYCI)
    print('along z-axis:\n', dipZCI)
elif debug == 4:
    for i in range(NStates):
        print('dipole moment (z) of CI state {}  = {} a.u.  '.format(i+1, dipZCI[i,i]))
debug = 0

**Writing read and computed CI data to `tdm_tdcis.txt` for TDCI calculation** 

In [27]:
# this part is written by bfh and is mostly unchanged

# have now calculated the Transition dipole moments. time to get atom coordinates and dump data
#  to file so TD_CIS.py can read it in. - bfh

outname = 'tdm_tdci.txt' # name of the output file, to be read by TDCI script

tdm = open(outname, 'w')
tdm.write('===========================================================================================\n')
tdm.write('  This file contains the information one needs to run a time-dependent CASSCF simulation.  \n')
tdm.write('===========================================================================================\n')
tdm.write('An initial Gaussian calculation was used to calculate the CI eigenvalues and eigenvectors  \n')
tdm.write('as well as the CI state configurations. The program calc_tdm.py was then used to calculate \n')
tdm.write('the transition dipole moments between the CI states as well as the project of the CI states\n')
tdm.write('onto the molecular orbitals. This file was generated by calc_tdm.py as input to TD_CIS.py. \n')
tdm.write(' -BFH 9 May 2014                                                                           \n')

tdm.write('\n\n')

# parameters for the TDCI script

tdm.write('CI calculation parameters\n\n')
tdm.write(' title   = {}\n'.format(molecule+'_'+basis))
tdm.write(' method  = {}\n'.format(ci))
tdm.write(' NMOs    = {}\n'.format(NMOs))
tdm.write(' NCAS    = {}\n'.format(NCAS))
tdm.write(' NFRZ    = {}\n'.format(NFRZ))
tdm.write(' NOCC    = {}\n'.format(NOCC))
tdm.write(' NELECT  = {}\n'.format(NELECT))
tdm.write(' NAtoms  = {}\n'.format(NAtoms))
tdm.write(' NStates = {}\n'.format(NStates))

tdm.write('\n\n')

tdm.write('Nuclear dipole moments (a.u.) \n\n')
tdm.write(' x = {: 4.3f} \n'.format(nuc_dipx*DtoAU))
tdm.write(' y = {: 4.3f} \n'.format(nuc_dipy*DtoAU))
tdm.write(' z = {: 4.3f} \n'.format(nuc_dipz*DtoAU))

tdm.write('\n\n')

tdm.write('Setting up the atomic coordinates \n\n')
tdm.write(' Standard orientation: \n')
tdm.write(' ------------------------------------------------------------------- \n')
tdm.write(' Center   Atomic     Atomic            Coordinates (Angstroms)       \n')
tdm.write(' Number   Number      Type            X            Y          Z      \n')
tdm.write(' ------------------------------------------------------------------- \n')


coords = np.zeros([NAtoms,3], np.float64)
atom_info = np.zeros([NAtoms,3], np.float64)

for (n, line) in enumerate(log_lines):
    if (' Standard basis:' in line):
        for i in range(NAtoms):
            elements = log_lines[n + i + 6].split()
            atom_info[i,0] = float(elements[0])
            atom_info[i,1] = float(elements[1])
            atom_info[i,2] = float(elements[2])
            coords[i,0] = float(elements[3])
            coords[i,1] = float(elements[4])
            coords[i,2] = float(elements[5])
            tdm.write('  %-10s %-10s %-10s %-10s %-10s %-10s' % (atom_info[i][0], atom_info[i][1], atom_info[i][2], coords[i][0], coords[i][1], coords[i][2]) + "\n")

tdm.write('\n\n')

# input SCF energy- assume energy of first NState - bfh

tdm.write('SCF Energy \n\n')
tdm.write(' SCF Done: E(RHF) = ' + str(CI_E[0]) + '  Ha \n')

tdm.write('\n\n')

# Excited State energies - bfh

tdm.write('Excited state energies (eV) \n')
tdm.write('\n')
for i in range(1,NStates):
    prt_E = (CI_E[i] - CI_E[0])*AUtoEV
    tdm.write(' Excited State {:3d} \t {:4.6f}\n'.format(i, prt_E))

tdm.write('\n\n')

# CI electric dipole moments - bfh

tdm.write('CI electric dipole moments (a.u.) \n\n')
tdm.write(' CI state and transition dipole moments\n')
tdm.write(' state I  state J \t      X \t\t      Y \t\t      Z \t\t   Dip. S. \t\t    Osc.\n' )
for i in range(NStates):
    for j in range(NStates):
        wij = abs(CI_E[i] - CI_E[j])
        fij = (2/3) * wij * ((dipXCI[i,j])**2 + (dipYCI[i,j])**2 + (dipZCI[i,j])**2)
        tdm.write('   {} \t    {} \t\t  {:3.7f}  \t  {:3.7f}  \t  {:3.7f}  \t  {:3.7f}  \t  {:3.7f}\n'.format(i, j, dipXCI[j,i], dipYCI[j,i], dipZCI[j,i], 0.0, fij))

tdm.write('\n\n')

# Finally, need to write out information on CI configs and MOs occupations - bfh
tdm.write('Output CI Vectors \n\n')
tdm.write(' CI state       CI coeffs 1...N \n')
for i in range(NStates):
    tdm.write('  %4d \t\t' % i )
    for j in range(NStates):
        tdm.write('%6.6f\t' % CI_vect[i,j])
    tdm.write('\n')

tdm.write('\n\n')

#note, want state # then number for each orbital. something like
#   state       MO 1     MO 2    MO 3     MO 4
#     1         1.95     1.95    0.05     0.05
# - bfh

mo_weight = np.zeros([NCAS], np.float64)
tdm.write('Orbital electron configuration weighting for CI States \n\n')
tdm.write(' CI State \t\t Orbital Weights 1...N  \n')
for i in range(NStates):
    for k in range(NCAS):
        mo_weight[k] = config[i,k]
    tdm.write('  {:3d}\t\t'.format(i))
    for k in range(NCAS):
        if (k == (NCAS-1)):
            tdm.write('{:4.6f}\n'.format(mo_weight[k]))
        else:
            tdm.write('{:4.6f}\t'.format(mo_weight[k]))
        mo_weight[k] = 0

tdm.write('\n\n')

tdm.close()

In [43]:
for i in range(4):
    for j in range(4):
        if i > j:
            continue
            dens_file = 'dens_ci_'+str(j)+'_'+str(i)+'.npz'
            denmat = np.load(dens_file)['arr_0']
            #print(2*denmat)
        else:
            print(i,j)
            dens_file = 'dens_ci_'+str(i)+'_'+str(j)+'.npz'
            denmat = np.load(dens_file)['arr_0']
            print(2*denmat)

0 0
[[0.105103 0.36506 ]
 [0.36506  1.50179 ]]
0 1
[[ 2.81356  -1.515568]
 [-1.515568  0.818452]]
0 2
[[ 2.81356  -1.515568]
 [-1.515568  0.818452]]
0 3
[[ 2.81356  -1.515568]
 [-1.515568  0.818452]]
1 1
[[ 1.408234 -0.758214]
 [-0.758214  1.408234]]
1 2
[[0. 0.]
 [0. 0.]]
1 3
[[0. 0.]
 [0. 0.]]
2 2
[[ 1.30604  -1.124132]
 [-1.124132  1.904458]]
2 3
[[ 0.0346278  0.185654 ]
 [-0.0221548 -0.1674032]]
3 3
[[ 2.81356  -1.515568]
 [-1.515568  0.818452]]
